# Running Eval14/15 in a QCrBox Docker container
Eval15 is a program to integrate crystallographic frames and evaluate the intensities of the reflections. The executable must be obtained separately from the [Eval website](http://www.crystal.chem.uu.nl/distr/eval/).

## Import and setting up folders / paths

We start by downloading some example files.

In [1]:
import zipfile
import urllib.request
from pathlib import Path

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper

example_frames = Path('./input_files/Ylid_OD_Images.zip')
example_input_files = Path('./input_files/eval_input.zip')

for zip_path in (example_frames, example_input_files):
    if not zip_path.exists():
        url = f'https://github.com/QCrBox/QCrBoxExamples/raw/main/Eval/{zip_path.name}'
        urllib.request.urlretrieve(url, zip_path)


We create an example folder to try out the Eval functionality.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_eval'
)
```

In [2]:
pathhelper = QCrBoxPathHelper.from_dotenv(
    '.env.dev',
    'examples_eval'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

frames_zip = './input_files/Ylid_OD_Images.zip'


## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [4]:
qcrbox = QCrBoxWrapper.from_server_addr('127.0.0.1', 11000)

In [5]:
qcrbox.application_dict

2024-08-22T19:48:22.933026Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


{'Eval14/15': <Eval14/15>}

Check that Eval14/15 is available in the application dict you just evaluated and then select it in the next cell.

In [6]:
eval1x = qcrbox.application_dict['Eval14/15']


2024-08-22T19:48:35.828890Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


As always we can get the list of available commands using pythons buildin help function.

In [7]:
help(eval1x)

Help on QCrBoxApplication in module qcrbox_wrapper.qcrbox_application object:

class QCrBoxApplication(builtins.object)
 |  QCrBoxApplication(application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |
 |  Represents an application in QCrBox packaged in its own container.
 |
 |  Methods defined here:
 |
 |  __init__(self, application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |      Initializes the QCrBoxApplication instance.
 |
 |      Parameters
 |      ----------
 |      application_spec: sql_models_NEW_v2.ApplicationSpecWithCommands
 |          The application spec as returned by the API endpoint `/applications`.
 |
 |  __repr__(self) -> str
 |      Return repr(self).
 |
 |  interactive_session(self, **kwargs)
 |
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |
 |  

## Starting an interactive session.
Currently the implementation of eval within QCrBox assumes that you first integrate your frames interactively following along [the thaumatin example from the Eval website](http://www.crystal.chem.uu.nl/distr/eval/documentation/ccd/doclib/example/thaumatin/index.html). 

A set of frames to use as example is included in `input_files/Ylid_OD_Images.zip`. So let us create a new folder and unpack the images to that folder.

In [8]:
# create a new folder for this example
folder_interact = path_local / 'run_interactive'
folder_interact.mkdir(exist_ok=True)

with zipfile.ZipFile(frames_zip, 'r') as zip_obj:
    zip_obj.extractall(folder_interact)

Now let us start an interactive session. Once we run the `interactive` command, two things happen: Firstly, a new browser window with the opens up, where a command line interface opens in the folder of your choice. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done following the example. Subsequently QCrBox will try to run a few programs to produce a cif file with the output via Evals output to SADABS' .sad format. There is no cif file at the beginning of an integration. As such there cannot be an `input_cif_path`. Instead we need to pass the folder where we want to work that contains the frames. You can now try to follow along the [the thaumatin example from the Eval website](http://www.crystal.chem.uu.nl/distr/eval/documentation/ccd/doclib/example/thaumatin/index.html) using the Ylid frames.

Here are the changes to the example:

#### proteinsetup
The unit cell of Ylid is about a=5 Ang, b=8 Ang, c=18 Ang, change the maximum volume to 1500 Ang.

#### renameimages
The images are already renamed so skip

#### scancheck
You need to run `scandb` first

#### view
Do not change the detectoroffset from the values in the frames. (But export a detalign.vic). The beamstop settings are 
 - `beamstop 0.0 -1.3`
 - `beamstopwidth 1.5`
 - `beamstopdiameter 4`
 - `beamstopangle 1`

Also set `resomax 0.79`, the number of peaks on a frame is 10

#### buildsearch / buildsearch=2
Be sure to set the maximum resolution and the number of peaks per frame correctly

#### buildeval15
Use focus type mirror and no polarisation

#### shellscript-2
Use `eval15all` instead by typing `eval15all` into the commandline

#### any
You do not need to set the pointgroup with pg

#### sadabs and shellscript-3
sadabs is not independent of eval and therefore not available within the container.

In [9]:
session = eval1x.interactive_session(
    work_folder=path_qcrbox / 'run_interactive',
    output_cif_path=path_qcrbox / 'run_interactive' / 'output.cif',
)
session.start()


2024-08-22T19:49:10.049717Z [debug    ] Starting interactive session for 'eval1x' extra={}
2024-08-22T19:49:10.051961Z [debug    ] self.prepare_calculation=None  extra={}
2024-08-22T19:49:10.079005Z [debug    ] self.run_calculation=<QCrBoxCalculation: 'qcrbox_calc_0x491c40c520714bc4ae1817c09c926dbe'> extra={}


gio: http://127.0.0.1/gui/eval1x: Operation not supported


In [10]:
session.close()

2024-08-22T19:50:18.369932Z [debug    ] Closing session                extra={}
2024-08-22T19:50:18.389196Z [debug    ] self.finalise_calculation=<QCrBoxCalculation: 'qcrbox_calc_0x7217aae8214f40e7834f7c14ecfd9622'> extra={}
2024-08-22T19:50:18.390543Z [debug    ] Done.                          extra={}


## Running an integration with slightly changed parameters

There is a basic way to run parts the data processing pipeline from the input files of an interactive session. The number of necessary files might be reduced in the future. This allows to reprocess frames that have been subject to Eval15 before. We can pull the necessary files from our interactive session from before to rerun the integration with a different resolution. Alternatively you can use the files in the `./input_files/eval_input.zip`. Let us create a folder again. You can set `use_files_from_zip` to `True` to use these.

In [11]:
use_files_from_zip = True  # Change to True to use settings files from zip

# create a new folder for this example
folder_integrate = path_local / 'run_integrate'
folder_integrate.mkdir(exist_ok=True)

with zipfile.ZipFile(frames_zip, 'r') as zip_obj:
    zip_obj.extractall(folder_integrate)

if use_files_from_zip:
    with zipfile.ZipFile('./input_files/eval_input.zip', 'r') as zip_obj:
        zip_obj.extractall(folder_integrate)
    source_folder = path_qcrbox / 'run_integrate'
else:
    source_folder = path_qcrbox / 'run_interactive'

We can now start the integration using the output files from either the zip archive or the interactive session. You might notice that we pass a path to a cif file as the `rmat_file_path` if it contains the required entries. An rmat file will work as well. The integration will probably take a bit of time. You can follow the progress within the `ic` folder.

In [12]:
calc2 = eval1x.integrate(
    work_folder=path_qcrbox / 'run_integrate',
    output_cif_path=path_qcrbox / 'run_integrate' / 'output.cif',
    rmat_file_path=source_folder / 'output.cif',
    beamstop_file_path=source_folder / 'beamstop.vic',
    detalign_file_path=source_folder / 'detalign.vic',
    maximum_res=0.79,
    minimum_res=50.0,
    box_size=1.2,
    box_depth=5,
    maximum_duration=5.0,
    min_refln_in_box=1000,
    pic_dir=source_folder / 'ic'
)

calc2.wait_while_running(1.0)

In [13]:
calc2.status

<CalculationStatusEnum.SUCCESSFUL: 'successful'>